In [ ]:
import glob

import numpy as np
import scipy.ndimage
import scipy.stats
%pylab inline

In [ ]:
fluct_mean = []
fluct_std = []


for i, p in enumerate(glob.glob('/global/homes/b/bzh/clamato-xcorr/data/xcorr/mock/skewers/dachshund/map*.bin')):
    fluct = np.fromfile(p, dtype=np.float64).reshape(68, 56, 1000)
    fluct_mean.append(np.mean(fluct))
    fluct_std.append(np.std(fluct))
    if i % 10 == 0:
        plt.figure(figsize=(10, 2))
        plt.imshow(fluct[34, :, :])
        plt.gca().set_aspect('equal')
        plt.show()

In [ ]:
plt.hist(fluct_mean)

In [ ]:
plt.hist(fluct_std)

# Compare 3 absorption realizations for same footprint

In [ ]:
pearsons = []
for i in range(1, 64):
    fluct_list = [np.fromfile(f'/global/homes/b/bzh/clamato-xcorr/data/xcorr/mock/skewers/dachshund/map_dachshund_mock_{i+j}.bin') for j in [0, 63, 126]]
    pearsons += [scipy.stats.pearsonr(fluct_list[0], fluct_list[1])[0], scipy.stats.pearsonr(fluct_list[1], fluct_list[2])[0], scipy.stats.pearsonr(fluct_list[0], fluct_list[2])[0]]

In [ ]:
print(len(pearsons))
plt.hist(pearsons);

# Check map 1 (corner) against true density

In [ ]:
fluct_corner = np.fromfile('/global/homes/b/bzh/clamato-xcorr/data/xcorr/mock/skewers/dachshund/map_dachshund_mock_1.bin').reshape(68, 56, 1000)
# Use only the front half.
fluct_corner = fluct_corner[:, :, :500]

flux = (fluct_corner + 1) * 0.8
# Trim flux above 1 to 1.
flux[flux > 1] = 1
tau = -np.log(flux)
overdens = (tau / 0.226)**(1/1.5)

# 1 + delta.
plt.hist(overdens.flatten(), bins=100);
print(np.nanmedian(overdens.flatten()))
print(np.sum(np.isnan(overdens) / overdens.size))

In [ ]:
true_dens = np.load('/global/homes/b/bzh/projectdir/bzh/tweb-gal-align/BolshoiP Data/z2.5/BolshoiP_rsd_cic_n512_0094.npy')
true_npix = true_dens.shape[0]
true_dens /= np.mean(true_dens)
x_ub = int(np.around(34 * (true_npix / 250)))
y_ub = int(np.around(28 * (true_npix / 250)))
print(x_ub, y_ub)
cut_true_dens = true_dens[:x_ub, :y_ub]
del true_dens
cut_true_dens = scipy.ndimage.gaussian_filter(cut_true_dens, 2.5 * (true_npix / 250))

In [ ]:
plt.hist(cut_true_dens.flatten(), bins=100, range=(0, 5));

In [ ]:
zoomed_true_dens = scipy.ndimage.zoom(cut_true_dens, (68 / cut_true_dens.shape[0], 56 / cut_true_dens.shape[1], 500 / cut_true_dens.shape[2]))

In [ ]:
nan_mask = ~np.isnan(overdens.flatten())
print(scipy.stats.pearsonr(zoomed_true_dens.flatten()[nan_mask], overdens.flatten()[nan_mask]))

In [ ]:
plt.imshow(zoomed_true_dens[34, :, :], vmax=np.max(overdens))
plt.show()
plt.imshow(overdens[34, :, :])

In [ ]:
pearsons_2 = []

for i in range(1, 190):
    fluct_corner = np.fromfile(f'/global/homes/b/bzh/clamato-xcorr/data/xcorr/mock/skewers/dachshund/map_dachshund_mock_{i}.bin').reshape(68, 56, 1000)
    # Use only the front half.
    fluct_corner = fluct_corner[:, :, :500]

    flux = (fluct_corner + 1) * 0.8
    # Trim flux above 1 to 1.
    flux[flux > 1] = 1
    tau = -np.log(flux)
    overdens = (tau / 0.226)**(1/1.5)
    nan_mask = ~np.isnan(overdens.flatten())
    pearsons_2.append(scipy.stats.pearsonr(zoomed_true_dens.flatten()[nan_mask], overdens.flatten()[nan_mask])[0])

In [ ]:
plt.hist(pearsons_2)

In [ ]:
print(np.argsort(pearsons_2))